In [ ]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from pathlib import Path
import nbimporter

In [ ]:
import utility_fncs
from global_vars import *

In [ ]:
from openai import OpenAI
import google.generativeai

In [ ]:
load_dotenv(override=True)
openai_api_key=os.getenv('OPEN_API_KEY')
google_api_key=os.getenv('GOOGLE_API_KEY')
model_openai_4omini = 'gpt-4o-mini'
model_openai_4onano = 'gpt-4.1-nano'
model_openai_o3mini = 'o3-mini'
model_genimi = 'gemini-2.0-flash'
model_gemini_2_5_flash_4_17 = "gemini-2.5-flash-preview-04-17"
model_gemini_2_5_flash_5_20 = "gemini-2.5-flash-preview-05-20"
model_gemini_2_5_pro_preview='gemini-2.5-pro-preview-06-05'

In [ ]:
openai = OpenAI()
google.generativeai.configure()

In [ ]:
#gemini_via_openai_client = OpenAI(
#    api_key=google_api_key,
#    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
#)
#response = gemini_via_openai_client.chat.completions.create(
#    model=model_gemini_2_5_flash_4_17,
#    messages=prompts,
#)
#print(response.choices[0].message.content)

In [ ]:

def call_gpt(msg, filedata=None):
    #print(filedata)   
    prompt_gpt.append({"role": "user", "content":msg})
    if filedata: prompt_gpt.append({"role": "user", "content":utility_fncs.getFileContent(filedata)})
    completion = openai.chat.completions.create(
        model=model_openai_4onano,
        messages=prompt_gpt,
    )
    reply=completion.choices[0].message.content
    prompt_gpt.append({"role":"assistant", "content":reply})
    return reply
    

In [ ]:
prompt_google_gemini = []
gemini = google.generativeai.GenerativeModel(
        model_name = model_gemini_2_5_flash_5_20,    
        system_instruction=system_gemini_msg
    )

def upload_file_gemini(file_path):
    file = google.generativeai.upload_file(file_path)
    print(file.uri)
    return file
    
def call_google_gemini(msg, file_path=None):
    prompt_msg=[{"text": msg}]
            
    
    if file_path:
        file_up=upload_file_gemini(file_path)
        prompt_msg.append(
        {
                "file_data": {
                    "file_uri": file_up.uri,
                    "mime_type": utility_fncs.detect_mime_type(file_path)
                }
            },)
    prompt_google_gemini.append({"role": "user", "parts": prompt_msg})
    response = gemini.generate_content(prompt_google_gemini)
    prompt_google_gemini.append({"role":"model","parts":[{"text":response.text}]})
    return response.text


In [ ]:
def start_conv():
    print(f"Gemini: Hi\n")
    reply_google='Hi'
    for i in range(5):
        reply_gpt = call_gpt(reply_google)
        print(f"GPT: {reply_gpt}\n")
        reply_google = call_google_gemini(reply_gpt)
        print(f"Google: {reply_google}\n")
        

In [ ]:

    
def dispatch_ai_call(qt,file,model):
    
    if model=='GPT':
        return call_gpt(qt, file)
    elif model=='GEMINI':
        return call_google_gemini(qt,file)
    

In [ ]:
dispatch_ai_call("how are you. tell me a joke about this file","C:/Projects/llm_engg/llm_engineering/requirements.txt",'GEMINI')